In [ ]:
# DISCLAIMER: Aviso de isenção de responsabilidade
# Esse projeto é apenas um teste do meu aprendizado e os resultados não devem ser usados como conselhos financeiros ou recomendação de investimento. 
# Nesse projeto eu implemente um modelo LSTM simples, para tentar prever o preço de um ativo nos proximos 30 dias. 
# Modelos de machine learning possuem limitações que podem gerar erros significavos.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# DOWNLOAD DOS DADOS
ticker = "NVDA"   # mude aqui se quiser outro ativo
data = yf.download(ticker, start="2015-01-01", progress=False)
data = data[['Close', 'Volume']]
data.dropna(inplace=True)

# Indicadores simples
data['Return'] = data['Close'].pct_change()
data['MA7'] = data['Close'].rolling(window=7).mean()
data['MA21'] = data['Close'].rolling(window=21).mean()
data['Volatility'] = data['Return'].rolling(window=7).std()
data = data.dropna()

# NORMALIZAÇÃO
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# PREPARAR DADOS PARA LSTM
seq_len = 60
future_days = 30

X, y = [], []
for i in range(seq_len, len(scaled_data) - future_days):
    X.append(scaled_data[i-seq_len:i])
    y.append(scaled_data[i:i+future_days, 0])  # apenas o preço de fechamento
X, y = np.array(X), np.array(y)

# Split treino/teste mantendo ordem temporal
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# MODELO LSTM EMPILHADO
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(30)
])

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# TREINAMENTO
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# PREVISÃO MULTI-STEP REAL
last_sequence = scaled_data[-seq_len:]
last_sequence = np.expand_dims(last_sequence, axis=0)

predicted_scaled = model.predict(last_sequence)
predicted_prices = scaler.inverse_transform(
    np.concatenate([predicted_scaled[0].reshape(-1,1),
                    np.zeros((future_days, scaled_data.shape[1]-1))], axis=1)
)[:,0]

# PLOTAGEM
last_date = data.index[-1]
future_dates = pd.date_range(last_date, periods=future_days+1, freq='B')[1:]

plt.figure(figsize=(12,6))
plt.plot(data.index[-150:], data['Close'].iloc[-150:], label="Histórico", color='blue')
plt.plot(future_dates, predicted_prices, label="Previsão (30 dias)", color='orange')
plt.title(f"Previsão de {ticker} - LSTM (30 dias)")
plt.xlabel("Data")
plt.ylabel("Preço (R$)")
plt.legend()
plt.grid(True)
plt.show()

# MÉTRICAS DE ERRO
pred_test = model.predict(X_test)
rmse = np.sqrt(np.mean((pred_test - y_test)**2))
print(f" RMSE no conjunto de teste: {rmse:.6f}")

print("\n Previsão concluída!")
ultimo_preco_real = float(data['Close'].iloc[-1])
ultimo_preco_previsto = float(predicted_prices[-1])

print(f"Último preço real: R$ {ultimo_preco_real:.2f}")
print(f"Preço previsto para os proximos 30 dias: R$ {ultimo_preco_previsto:.2f}")